# Configs

In [2]:
import pandas as pd
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import random

In [18]:
# Configs

MAX_LEN = 128
data_path = './data/tfidf.csv'
candidates_num = 8 #标准话术数量

# 如果ConnectionError就把local_files_only改成True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',local_files_only=False,do_lower_case=True)

# Embed Text

In [6]:
df = pd.read_csv(data_path,index_col=0)
df.head()

,context,response,label_,standard
0,呃 来电 是 感谢 你 对 联通 的 支持 嘛 从 下个月 开始 每月 25 元 就 可以 ...,嗯 那 好 那 耽误 20 秒 感谢 你 的 支持 啊 每月 25 元 的 钻石 了 会员 ...,1,好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众...
1,嗯 那 好 那 耽误 20 秒 感谢 你 的 支持 啊 每月 25 元 的 钻石 了 会员 ...,嗯 那 好 那 这边 先 不 打扰 您 了 祝您 生活 愉快 再见 嗯,1,好 的 使用 中 您 可以 下载 中国联通 APP 查询 您 的 消费 情况 关注 最新 活...
2,好 的 来电 的话 先 感谢 你 对 联通 的 支持 下个月 开始 每月 25 元 就 可以...,让 大家 重新 建立 嗯 这边 重新 跟 你 说 一遍 嘛 就是说 这边 来电 的话 就是 ...,1,来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...
3,让 大家 重新 建立 嗯 这边 重新 跟 你 说 一遍 嘛 就是说 这边 来电 的话 就是 ...,那 你 下个月 收到 短信 就 可以 领取 这些 会员 的 了 那 这边 给 你 10 周年...,1,来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...
4,那 你 下个月 收到 短信 就 可以 领取 这些 会员 的 了 那 这边 给 你 10 周年...,好 的 那 这边 给 那 这边 的话 感谢 你 的 支持 的 从 下个月 开始 正式 入 会...,1,好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众...


In [9]:
!pip install fast_ml

In [10]:
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test
= train_valid_test_split(df, target = 'label_',train_size=0.8, valid_size=0.1, test_size=0.1)

## Fake Candidates (New)

In [11]:
after = list(df.standard.unique())
after

['好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众 号 沃 音乐 领取 就 好 了 \n 每月 25 元除 月租 外 一次性 扣取 立即 生效 自动 续订 \n 联通 还会 返还 您 40 元 话费 分 4 个 月 每月 返还 10 元 话费 会 在 下个月 10 号 前 到 账 \n 给 您 受理 好 收到 生效 短信 后 就 可以 放心 用 的 了 好 吧 ',
 '好 的 使用 中 您 可以 下载 中国联通 APP 查询 您 的 消费 情况 关注 最新 活动 福利 祝您 生活 愉快 再见 ',
 '来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 可以 领取 两个 VIP 会员 \n 因为 您 是 我们 联通 的 优质 用户 联通 还会 返还 您 40 元 话费 分 四个 月 每月 返还 10 元 希望 可以 帮 你 几个 月 希望 您 用 得 好多 多 支持 联通 \n 给 您 受理 好 收到 生效 短信 后 就 可以 放心 用 的 了 好 吧 ',
 '就 每个 月 25 元 这边 有 话费 40 元 返还 给 您 抵扣 月租 的 分 四月 每月 到 账 10 元 希望 可以 帮 您 几个 月 希望 您 用 得 好多 多 支持 联通 \n 受理 好 收到 生效 短信 后 就 可以 领取 用 的 了 好 吧 ',
 '没关系 啊 这边 也 是 可以 给 您 续 的 而且 我们 这个 是 有 二十多种 会员 您 每个 月 可以 任意 选 两个 用 每个 月 还 可以 换 不同 的 会员 来 使用 的 受理 好 收到 生效 短信 后 就 可以 领取 用 的 了 好 吧 ',
 '您好 这边 是 联通 客服 来电 的 请问 手机 尾号 是 你 在 用 的 对 吧 ',
 '就 耽误 您 一两分钟 时间 现在 有 会员 给 到 您 用 联通 还会 返还 40 元 话费 给 您 分 四月 每月 到 账 10 元 希望 可以 帮 您 几个 月 希望 您 用 得 好多 多 支持 联通 受理 好 收到 生效 短信 后 就 可以 放心 用 的 了 好 吧 ',
 '不会 影响 您 的 套餐 的 每月 25 元 联通 还会 返还 40 元 话费 给 您 分 四月 每月 到 账 10 元 希望

In [24]:
from copy import deepcopy

def make_candidates(X):
  fake_candidates = []
  new_labels = []
  for i in range(0,len(X)):
    temp = deepcopy(after)
    fake_candidates.append(X.reset_index().standard[i])
    temp.remove(X.reset_index().standard[i])
    assert len(temp) == len(after)-1
    fake_candidates.extend(temp)
    candidates = [1]
    candidates.extend([0]*(candidates_num-1))
    new_labels.extend(candidates)
  new_X = X.loc[X.index.repeat(8)]
  new_X.response = fake_candidates
  return new_X, new_labels

In [13]:
X_train, y_train = make_candidates(X_train)
X_valid, y_valid = make_candidates(X_valid)
X_test, y_test = make_candidates(X_test)

In [25]:
def vectorize(text,MAX_LEN=MAX_LEN, tokenizer=tokenizer):

    input_ids = []
    for t in text:
        # so basically encode tokenizing , mapping sentences to thier token ids after adding special tokens.
        encoded_sent = tokenizer.encode(
            t,  # Sentence which are encoding.
            add_special_tokens=True,  # Adding special tokens '[CLS]' and '[SEP]'
        )
        input_ids.append(encoded_sent)

    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="pre", padding="post")

    attention_masks = []
    for sent in input_ids:
        # Generating attention mask for sentences.
        #   - when there is 0 present as token id we are going to set mask as 0.
        #   - we are going to set mask 1 for all non-zero positive input id.
        att_mask = [int(token_id > 0) for token_id in sent]

        attention_masks.append(att_mask)

    return attention_masks, input_ids

In [26]:
from six import b
train = {}
train[b('y')] = y_train
val = {}
val[b('y')] = y_valid
test = {}
test[b('y')] = y_test

In [27]:
_, train_c = vectorize(X_train.context)
_, train_r = vectorize(X_train.response)

_, val_c = vectorize(X_valid.context)
_, val_r = vectorize(X_valid.response)

_, test_c = vectorize(X_test.context)
_, test_r = vectorize(X_test.response)

In [28]:
train[b('c')] = train_c.tolist()
train[b('r')] = train_r.tolist()

val[b('c')] = val_c.tolist()
val[b('r')] = val_r.tolist()

test[b('c')] = test_c.tolist()
test[b('r')] = test_r.tolist()

In [ ]:
input_data = (train,val,test)

# Pickle as pkl

In [ ]:
import pickle

with open('./models-release-1.5/PaddleNLP/dialogue_model_toolkit/deep_attention_matching/data/data.pkl','wb') as f:
    pickle.dump(input_data,f)